# Расчёт среднего времени ответа менеджера клиенту. Введение #

**Задача**: подсчитать среднее время ответа менеджера/пары менеджеров на первичное обращение клиента с учётом рабочего времени с 9:30 до 00:00.я.

## Imports ##

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import time, datetime

import psycopg2

In [2]:
HOSTNAME = '#####'
PORT = '#####'
USERNAME = '#####'
PASSWORD = '#####'
DATABASE = '#####

In [3]:
# подключение
conn_params = {
    'host': HOSTNAME,
    'port': PORT,
    'database': DATABASE,
    'user': USERNAME,
    'password': PASSWORD
}

conn = psycopg2.connect(**conn_params)

In [4]:
# создание фреймов
chat = pd.read_sql('SELECT * FROM test.chat_messages', conn)
managers = pd.read_sql('SELECT * FROM test.managers', conn)
rops = pd.read_sql('SELECT * FROM test.rops', conn)

/tmp/ipykernel_46466/4292642281.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  chat = pd.read_sql('SELECT * FROM test.chat_messages', conn)
/tmp/ipykernel_46466/4292642281.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  managers = pd.read_sql('SELECT * FROM test.managers', conn)
/tmp/ipykernel_46466/4292642281.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  rops = pd.read_sql('SELECT * FROM test.rops', conn)


In [5]:
display(chat.shape)
display(managers.shape)
display(rops.shape)

(18929, 5)

(26, 3)

(3, 2)

In [6]:
# проверяем подключение
display(chat.head(1))
print('managers')
display(managers.head(1))
print('rops')
display(rops.head(1))

,message_id,type,entity_id,created_by,created_at
0,"""01jb7da570sf4f65xdf0ptvv9q""",incoming_chat_message,37556493,0,1730046924


managers


,mop_id,name_mop,rop_id
0,6645315,Гюнель и Илина,1


rops


,rop_id,rop_name
0,1,Катя РОП


In [7]:
# закрытие коннектора
conn.close()

## Предобработка ##

Учитывая, что перед нами собранный внутри компании датасет, то мы предполагаем, что данные либо чистые изначалльно, либо уже очищенные, однако стоит проверить основную информацию и распределение времени в столбце *created_at*

In [8]:
display(chat.info())
print('-'*50)
display(chat.created_at.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18929 entries, 0 to 18928
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   message_id  18929 non-null  object
 1   type        18929 non-null  object
 2   entity_id   18929 non-null  int64 
 3   created_by  18929 non-null  int64 
 4   created_at  18929 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 739.5+ KB


None

--------------------------------------------------


count    1.892900e+04
mean     1.729988e+09
std      4.640295e+04
min      1.729890e+09
25%      1.729946e+09
50%      1.729969e+09
75%      1.730034e+09
max      1.730063e+09
Name: created_at, dtype: float64

> Мы видим, что у нас отсутствуют пропуски, столбцы имеют корректный тип, а время имеет адекватное распределение, а низкое стандартное отклонение показывает, что все значения распределены достаточно близко к друг другу.

In [9]:
# копируем датасеты, чтобы не подключаться к бд снова
chat_c = chat.copy()
managers_c = managers.copy()
rops_c = rops.copy()

In [10]:
# проверим на сходимость
print('Расхождений: \n', chat[chat_c != chat].sum())

Расхождений: 
 message_id      0
type            0
entity_id     0.0
created_by    0.0
created_at    0.0
dtype: object


In [11]:
# проверим датасет на отсутствие случаев, 
# при которых сообщения имеют тип "исходящее" и не были созданы оператором

chat_c[(chat_c.type == 'outgoing_chat_message') & (chat_c.created_by == 0)].count()

message_id    173
type          173
entity_id     173
created_by    173
created_at    173
dtype: int64

In [12]:
# уберем некорректные значения в парах outgoing_chat_msgs в пересечении с created_by == 0
chat_c = chat_c[~((chat_c['created_by'] == 0) & (chat_c['type'] == 'outgoing_chat_message'))]

In [13]:
# проверим результат
chat_c[(chat_c.type == 'outgoing_chat_message') & (chat_c.created_by == 0)].count()

message_id    0
type          0
entity_id     0
created_by    0
created_at    0
dtype: int64

Согласно заданию, мы должны учитывать только первичный ответ менеджера. Для получения нужного датасета необходимо воспользоваться сортировкой по *id* чата и времени его создания, чтобы первым сообщением в чате шло сообщение клиента, а вторым - менеджера.

In [14]:
chat_c.sort_values(by=['entity_id', 'created_at']).head()

,message_id,type,entity_id,created_by,created_at
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,1729955966
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",outgoing_chat_message,35810509,10262493,1729956065
6430,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,1729956886
6639,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,1729957006
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,1730050716


Мы видим, что у одинаковых entity_id появился чёткий порядок. Сохраним отсортированный датасет в новую переменную. 

In [15]:
# отсортированный датасет
chat_c_sorted = chat_c.sort_values(by=['entity_id', 'created_at'])

# переведем unixtime в datetime c указанием часового пояса Мск
chat_c_sorted['created_at'] = chat_c_sorted['created_at'].apply(lambda x: pd.Timestamp(x, unit='s', tz='Europe/Moscow'))

chat_c_sorted.head()

,message_id,type,entity_id,created_by,created_at
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,2024-10-26 18:19:26+03:00
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",outgoing_chat_message,35810509,10262493,2024-10-26 18:21:05+03:00
6430,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,2024-10-26 18:34:46+03:00
6639,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,2024-10-26 18:36:46+03:00
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,2024-10-27 20:38:36+03:00


In [16]:
# используем функцию shift() - аналог sql-ного LAG() и получим время предыдущего ответа
chat_c_sorted['manager_answer_time'] = chat_c_sorted.groupby('entity_id')['created_at'].shift(-1)
chat_c_sorted['prev_type'] = chat_c_sorted.groupby('entity_id')['type'].shift()

> выделим типы сообщений, выделив их следующим образом
> - обращение клиента - 0
> - первичный ответ - 1
> - остальные типы оставим прежними

In [17]:
# добавляем новый пустой столбец
chat_c_sorted['upd'] = None

In [18]:
# выделяем первое сообщение клиента
chat_c_sorted.loc[(chat_c_sorted.created_by == 0) & 
               (chat_c_sorted.prev_type.isnull()) & 
               (chat_c_sorted.prev_type.shift(-1) == 'incoming_chat_message'), 'upd'] = 0

In [19]:
chat_c_sorted.head(10)

,message_id,type,entity_id,created_by,created_at,manager_answer_time,prev_type,upd
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,2024-10-26 18:19:26+03:00,2024-10-26 18:21:05+03:00,NaN,0
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",outgoing_chat_message,35810509,10262493,2024-10-26 18:21:05+03:00,2024-10-26 18:34:46+03:00,incoming_chat_message,None
6430,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,2024-10-26 18:34:46+03:00,2024-10-26 18:36:46+03:00,outgoing_chat_message,None
6639,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,2024-10-26 18:36:46+03:00,NaT,incoming_chat_message,None
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,2024-10-27 20:38:36+03:00,2024-10-27 20:42:07+03:00,NaN,0
17852,"""01jb7h4ad8weqjx1ynrh7s8k25""",outgoing_chat_message,35945535,10262493,2024-10-27 20:42:07+03:00,NaT,incoming_chat_message,None
14805,"""01jb727x90ga4ncv4m705rh0rm""",incoming_chat_message,36099337,0,2024-10-27 16:21:56+03:00,2024-10-27 16:25:47+03:00,NaN,0
14833,"""01jb72ezmaacke29h682geasez""",outgoing_chat_message,36099337,11241942,2024-10-27 16:25:47+03:00,2024-10-27 16:27:14+03:00,incoming_chat_message,None
14989,"""01jb72hktgm0tvdpwk4a17dadz""",incoming_chat_message,36099337,0,2024-10-27 16:27:14+03:00,2024-10-27 16:29:37+03:00,outgoing_chat_message,None
15039,"""01jb72nzkrzhcfhg9hfpbjt0vx""",outgoing_chat_message,36099337,11241942,2024-10-27 16:29:37+03:00,NaT,incoming_chat_message,None


In [20]:
# выделяем строки первого ответа менеджера
chat_c_sorted.loc[(chat_c_sorted.created_by == 0) & 
       (chat_c_sorted.prev_type.isnull()) & 
       (chat_c_sorted.prev_type.shift(-1) == 'incoming_chat_message'), 'upd'] = 1

In [21]:
# создадим новый столбец для сдвига id менеджера
chat_c_sorted['created_upd'] = chat_c_sorted.groupby('entity_id')['created_by'].shift(-1)
chat_c_sorted.head()

,message_id,type,entity_id,created_by,created_at,manager_answer_time,prev_type,upd,created_upd
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,2024-10-26 18:19:26+03:00,2024-10-26 18:21:05+03:00,NaN,1,10262493.0
4102,"""01jb4pncn7gm1re1zdxkzhnp6s""",outgoing_chat_message,35810509,10262493,2024-10-26 18:21:05+03:00,2024-10-26 18:34:46+03:00,incoming_chat_message,None,0.0
6430,"""01jb4qedfgxnd2yqr4w26hs4q5""",incoming_chat_message,35810509,0,2024-10-26 18:34:46+03:00,2024-10-26 18:36:46+03:00,outgoing_chat_message,None,10262493.0
6639,"""01jb4qj2skc2wjfdm8qaf69yjh""",outgoing_chat_message,35810509,10262493,2024-10-26 18:36:46+03:00,NaT,incoming_chat_message,None,NaN
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,2024-10-27 20:38:36+03:00,2024-10-27 20:42:07+03:00,NaN,1,10262493.0


In [22]:
# фильтруем таблицу по нескольким условиям, оставляя только строки 
# с уникальным entity_id, id менеджера и временем ответа 
chat_c_sorted = (
    chat_c_sorted[
    (chat_c_sorted.created_upd > 0) &
    (chat_c_sorted.manager_answer_time.notnull()) &
    (chat_c_sorted.prev_type.isnull())
    ]
)
chat_c_sorted                 

,message_id,type,entity_id,created_by,created_at,manager_answer_time,prev_type,upd,created_upd
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,2024-10-26 18:19:26+03:00,2024-10-26 18:21:05+03:00,NaN,1,10262493.0
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,2024-10-27 20:38:36+03:00,2024-10-27 20:42:07+03:00,NaN,1,10262493.0
14805,"""01jb727x90ga4ncv4m705rh0rm""",incoming_chat_message,36099337,0,2024-10-27 16:21:56+03:00,2024-10-27 16:25:47+03:00,NaN,1,11241942.0
9089,"""01jb4th8j8zztbcz0c68xq5pkf""",incoming_chat_message,36333761,0,2024-10-26 19:28:45+03:00,2024-10-26 19:33:28+03:00,NaN,1,10262485.0
11026,"""01jb6b07drbamdejg17n9qag80""",incoming_chat_message,36373713,0,2024-10-27 09:35:47+03:00,2024-10-27 09:37:38+03:00,NaN,1,7996770.0
...,...,...,...,...,...,...,...,...,...
17087,"""01jb7r0r3g19cjq8e1n8cczykk""",incoming_chat_message,37557101,0,2024-10-27 22:42:30+03:00,2024-10-27 22:54:03+03:00,NaN,1,11448610.0
18214,"""01jb7s4ngr7gewvby6fdgk9ae1""",incoming_chat_message,37557137,0,2024-10-27 23:02:07+03:00,2024-10-27 23:12:03+03:00,NaN,1,10262513.0
18354,"""01jb7sa788qvqs419jfckh4b3t""",incoming_chat_message,37557149,0,2024-10-27 23:05:09+03:00,2024-10-27 23:33:40+03:00,NaN,1,10465254.0
18252,"""01jb7sndn0b70srwkc6xjd15c0""",incoming_chat_message,37557161,0,2024-10-27 23:11:16+03:00,2024-10-27 23:33:56+03:00,NaN,1,10262493.0


> Подготовив наш датасет и очистив его от ненужных строк, необходимо задать учет время для окончательно подсчёта времени ответа. Согласно условию, рабочий день начинается в 9:30 и заканчивается в 00:00. При этом сообщения, отправленные ночью, также должны учитываться.

## Обработка времени и сведение итоговой таблицы ##

In [23]:
# создадим функцию условного расчёта для столбца
def worktime_calculate(time_col1, time_col2, row):
    start_time = pd.to_datetime('9:30')
    end_time = pd.to_datetime('00:00') + pd.Timedelta(days=1) # сдвиг учёта начала след. дня
    
    t1 = time_col1[row]
    t2 = time_col2[row]
    
    # Если значение t1 меньше 9:30, сдвигаем его на 9:30
    if t1.time() < start_time.time():
        t1 = t1.replace(hour=9, minute=30, second=0, microsecond=0)
        
    # Если значение t2 меньше 9:30, то фиксируем его на 9:30 следующего рабочего дня
    if t2.time() < start_time.time():
        t2 = t2.replace(hour=9, minute=30, second=0, microsecond=0) + pd.Timedelta(days=1)
        
    # Если t2 меньше t1, возвращаем None
    if t2 < t1:
        return None
        
    # Возвращаем разницу во времени
    return (t2 - t1).total_seconds() / 3600  # Возвращаем разницу в часах

In [24]:
def time_difference(data, time_col1, time_col2):
    # Преобразуем столбцы времени в формат datetime
    data[time_col1] = pd.to_datetime(data[time_col1])
    data[time_col2] = pd.to_datetime(data[time_col2])

    # начало и конец рабочего дня
    start_time = pd.to_datetime('09:30')
    end_time = pd.to_datetime('00:00') + pd.Timedelta(days=1) # сдвиг учёта начала след. дня

    def worktime_calculate(row):
        t1 = row[time_col1]
        t2 = row[time_col2]
        ''' Определим условия:
            - если значение t1 меньше 9:30, сдвигаем его на 9:30
            - если значение t2 меньше 9:30, то фиксируем его на 9:30 следующего рабочего дня
            - если t2 меньше t1, возвращаем None '''
        if t1.time() < start_time.time():
            t1 = t1.replace(hour=9, minute=30, second=0, microsecond=0)
        if t2.time() < start_time.time():
            t2 = t2.replace(hour=9, minute=30, second=0, microsecond=0) + pd.Timedelta(days=1)
        if t2 < t1:
            return None

        # возвращаем разницу в минутах
        return (t2 - t1).total_seconds() / 60  

    # применение к каждой строке входного фрейма
    data['answer_time_diff'] = data.apply(worktime_calculate, axis=1)

    return data['answer_time_diff']

In [25]:
chat_c_sorted['answer_time_diff'] = time_difference(chat_c_sorted, 
                                'created_at',
                                'manager_answer_time')

In [26]:
chat_c_sorted

,message_id,type,entity_id,created_by,created_at,manager_answer_time,prev_type,upd,created_upd,answer_time_diff
3107,"""01jb4pjb1g22ts7z6arcjv11w6""",incoming_chat_message,35810509,0,2024-10-26 18:19:26+03:00,2024-10-26 18:21:05+03:00,NaN,1,10262493.0,1.650000
17586,"""01jb7gxwb0sepv9vtekrgj3v6d""",incoming_chat_message,35945535,0,2024-10-27 20:38:36+03:00,2024-10-27 20:42:07+03:00,NaN,1,10262493.0,3.516667
14805,"""01jb727x90ga4ncv4m705rh0rm""",incoming_chat_message,36099337,0,2024-10-27 16:21:56+03:00,2024-10-27 16:25:47+03:00,NaN,1,11241942.0,3.850000
9089,"""01jb4th8j8zztbcz0c68xq5pkf""",incoming_chat_message,36333761,0,2024-10-26 19:28:45+03:00,2024-10-26 19:33:28+03:00,NaN,1,10262485.0,4.716667
11026,"""01jb6b07drbamdejg17n9qag80""",incoming_chat_message,36373713,0,2024-10-27 09:35:47+03:00,2024-10-27 09:37:38+03:00,NaN,1,7996770.0,1.850000
...,...,...,...,...,...,...,...,...,...,...
17087,"""01jb7r0r3g19cjq8e1n8cczykk""",incoming_chat_message,37557101,0,2024-10-27 22:42:30+03:00,2024-10-27 22:54:03+03:00,NaN,1,11448610.0,11.550000
18214,"""01jb7s4ngr7gewvby6fdgk9ae1""",incoming_chat_message,37557137,0,2024-10-27 23:02:07+03:00,2024-10-27 23:12:03+03:00,NaN,1,10262513.0,9.933333
18354,"""01jb7sa788qvqs419jfckh4b3t""",incoming_chat_message,37557149,0,2024-10-27 23:05:09+03:00,2024-10-27 23:33:40+03:00,NaN,1,10465254.0,28.516667
18252,"""01jb7sndn0b70srwkc6xjd15c0""",incoming_chat_message,37557161,0,2024-10-27 23:11:16+03:00,2024-10-27 23:33:56+03:00,NaN,1,10262493.0,22.666667


In [27]:
# передадим в новую переменную наш конечный датасет с агрегацией по id оператора и ср. времени ответа
chat_complete = chat_c_sorted.groupby('created_upd')['answer_time_diff'].mean().sort_values()
chat_complete

created_upd
6780177.0      3.422727
7417617.0      6.487500
8558094.0      6.965556
11241942.0     8.997222
10465254.0     9.337847
10262497.0     9.942029
7408305.0     11.193333
11394126.0    12.380000
11550890.0    13.027778
11394134.0    20.646667
6645315.0     28.908333
10262485.0    30.659259
11394130.0    34.708108
6744792.0     36.823689
11188802.0    38.914198
10262493.0    42.135556
11490270.0    43.119207
11448610.0    50.221816
11490278.0    52.428571
10465690.0    56.757092
10262513.0    69.147222
7996770.0     78.521164
Name: answer_time_diff, dtype: float64

In [28]:
# трансформируем итоговую таблицу в датафрейм
chat_complete = pd.DataFrame({'mop_id': chat_complete.index, 
                              'avg_answer_time': chat_complete.values})

In [29]:
chat_complete.head()

,mop_id,avg_answer_time
0,6780177.0,3.422727
1,7417617.0,6.487500
2,8558094.0,6.965556
3,11241942.0,8.997222
4,10465254.0,9.337847


In [30]:
# объединим таблицу с таблицей сотрудников для получения имён последних
chat_complete = chat_complete.merge(managers_c, on='mop_id')

In [31]:
chat_complete.head()

,mop_id,avg_answer_time,name_mop,rop_id
0,6780177.0,3.422727,Даша и Даша,2
1,7417617.0,6.487500,Порхачева Полина,3
2,8558094.0,6.965556,Алина и Юля,3
3,11241942.0,8.997222,Даша и Карина,3
4,10465254.0,9.337847,Настя и Даша,1


In [32]:
# то же самое сделаем и с руководителями
chat_complete['rop_id'] = chat_complete.rop_id.astype('int64')
chat_complete = chat_complete.merge(rops_c, on='rop_id')

In [33]:
try:
    chat_complete = chat_complete.drop('rop_id', axis = 1)
except:
    pass
chat_complete = chat_complete.sort_values(by='avg_answer_time')
chat_complete['mop_id'] = chat_complete.mop_id.astype('int32')
chat_complete['avg_answer_time'] = chat_complete['avg_answer_time'].astype('int32')
chat_complete = chat_complete.reindex(columns=['mop_id', 'name_mop', 'avg_answer_time', 'rop_name'])
chat_complete

,mop_id,name_mop,avg_answer_time,rop_name
0,6780177,Даша и Даша,3,Полина РОП
6,7417617,Порхачева Полина,6,Эля РОП
7,8558094,Алина и Юля,6,Эля РОП
8,11241942,Даша и Карина,8,Эля РОП
15,10465254,Настя и Даша,9,Катя РОП
1,10262497,Мария и Соня,9,Полина РОП
9,7408305,Влада и Настя,11,Эля РОП
10,11394126,Полина Мирзоян,12,Эля РОП
11,11550890,Ангелина Милованова,13,Эля РОП
12,11394134,Софья Боднар,20,Эля РОП


In [34]:
# сохраняем полученный датасет для дальнейшего импорта в csv-файл
chat_complete.to_csv('avg_managers_response_time.csv')

## Выводы ##

В ходе нашей работы мы подсчитали среднее время первичного ответа менеджера на запрос клиента с учётом рабочего времени с 9:30 до 00:00 и создали готовый набор данных, который можно использовать для дальнейшей работы.